In [12]:
from pathlib import Path
from datetime import datetime
import json

import torch
from PIL import Image
from transformers import AutoImageProcessor, AutoModelForImageClassification

In [13]:
old_model_name = "pittawat/vit-base-letter"
new_model_name = "pittawat/vit-base-uppercase-english-characters"

In [14]:
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")

In [23]:
def evaluate(model_name):
    TEST_FOLDER_PATH = Path("test")
    processor = AutoImageProcessor.from_pretrained(model_name)
    model = AutoModelForImageClassification.from_pretrained(model_name).to(device)

    summary = {
        "model_name": model_name,
        "correct": 0,
        "total": 0,
        "accuracy": 0,
        "predictions": [],
        "created_at": str(datetime.now())
    }

    for folder in TEST_FOLDER_PATH.iterdir():
        if not folder.is_dir():
            continue
        current_target = folder.name
        for image_path in folder.iterdir():
            if not image_path.is_file() or not image_path.suffix in [".jpg", ".png"]:
                continue
            image = Image.open(image_path).convert("RGB")
            inputs = processor(image, return_tensors="pt").to(device)
            outputs = model(**inputs)
            print(f"Image: {image_path}")
            print(f"Predicted: {outputs.logits.argmax(-1)}")
            print(f"Label: {ord(current_target) - 65}")
            print("True: ", ord(current_target) - 65 == outputs.logits.argmax(-1))

            summary["predictions"].append({
                "image_path": str(image_path),
                "predicted": outputs.logits.argmax(-1).item(),
                "label": ord(current_target) - 65,
                "true": (ord(current_target) - 65 == outputs.logits.argmax(-1)).item()
            })
            summary["total"] += 1

            if ord(current_target) - 65 == outputs.logits.argmax(-1):
                summary["correct"] += 1

    summary["accuracy"] = summary["correct"] / summary["total"]
    with open(f"{model_name.replace('/', '_')}.json", "w") as f:
        json.dump(summary, f, indent=4)

In [24]:
evaluate(old_model_name)

Image: test/R/4.png
Predicted: tensor([4], device='mps:0')
Label: 17
True:  tensor([False], device='mps:0')
Image: test/R/2.png
Predicted: tensor([5], device='mps:0')
Label: 17
True:  tensor([False], device='mps:0')
Image: test/R/3.png
Predicted: tensor([7], device='mps:0')
Label: 17
True:  tensor([False], device='mps:0')
Image: test/R/1.png
Predicted: tensor([7], device='mps:0')
Label: 17
True:  tensor([False], device='mps:0')
Image: test/R/0.png
Predicted: tensor([17], device='mps:0')
Label: 17
True:  tensor([True], device='mps:0')
Image: test/U/4.png
Predicted: tensor([20], device='mps:0')
Label: 20
True:  tensor([True], device='mps:0')
Image: test/U/2.png
Predicted: tensor([20], device='mps:0')
Label: 20
True:  tensor([True], device='mps:0')
Image: test/U/3.png
Predicted: tensor([20], device='mps:0')
Label: 20
True:  tensor([True], device='mps:0')
Image: test/U/1.png
Predicted: tensor([20], device='mps:0')
Label: 20
True:  tensor([True], device='mps:0')
Image: test/U/0.png
Predicte

In [25]:
evaluate(new_model_name)

Image: test/R/4.png
Predicted: tensor([17], device='mps:0')
Label: 17
True:  tensor([True], device='mps:0')
Image: test/R/2.png
Predicted: tensor([17], device='mps:0')
Label: 17
True:  tensor([True], device='mps:0')
Image: test/R/3.png
Predicted: tensor([17], device='mps:0')
Label: 17
True:  tensor([True], device='mps:0')
Image: test/R/1.png
Predicted: tensor([17], device='mps:0')
Label: 17
True:  tensor([True], device='mps:0')
Image: test/R/0.png
Predicted: tensor([17], device='mps:0')
Label: 17
True:  tensor([True], device='mps:0')
Image: test/U/4.png
Predicted: tensor([20], device='mps:0')
Label: 20
True:  tensor([True], device='mps:0')
Image: test/U/2.png
Predicted: tensor([20], device='mps:0')
Label: 20
True:  tensor([True], device='mps:0')
Image: test/U/3.png
Predicted: tensor([20], device='mps:0')
Label: 20
True:  tensor([True], device='mps:0')
Image: test/U/1.png
Predicted: tensor([20], device='mps:0')
Label: 20
True:  tensor([True], device='mps:0')
Image: test/U/0.png
Predicte